# `git2net` - Extracting and analysing co-editing relationships from *git* repositories

In this tutorial you will learn the basic steps required to obtain a co-editing relationship from a git repoitory using `git2net`.

## Prerequisits

This tutorial assumes you have `git2net` installed. In addition, it is recommended to create a folder for this tutorial as additional files will be downloaded to your local directory (if not specified otherwise).

## Repository Mining

To start, you will need to select and clone a git repository that you are interested in analysing. For the purpose of this tutorial, we will analyse the repository behind `git2net`&mdash;aiming to finally find a solution to the well-known chicken and egg problem.

The following lines will clone the `git2net` repository to your current working directory. To change this location, you can edit the path to the local directory stored in `local_directory`. The folder name of the repository is the name of the repository, which we store in `repo_name`.

In [1]:
import git
import os
import shutil

repo_url = 'https://github.com/gotec/git2net.git'
local_directory = '.'
repo_dir = 'git2net4analysis'

if os.path.exists(repo_dir):
    shutil.rmtree(repo_dir)

git.Git(local_directory).clone(repo_url, repo_dir)

''

Now that we have obtained a local copy of the repository, we can use `git2net` to obtain a database containing information on all commits and edits made to obtain the current state of the repository.

To do so, we use the `mine_git_repo` function. This function takes two required inputs as well as a number of optional inputs, some of which we will further explore later in this tutorial. Let's start with the required inputs. Here, we need to supply a path to the git repositoy that will be analysed. Below, this is done with the variable `repo_name`. In addition, `git2net` requires a path to the *sqlite* database that will be filled during the mining process. This path is provided as `sqlite_db_file`.

Note, that if no database exists on the supplied path, `git2net` will create a new database. If a database exists, `git2net` will check if the database was mined with the same setting and on the same repository and subsequently resume the mining process from wherever it was left off.

Let's try this out. Below we import `git2net` and point it to the path to which we cloned the database. In addition, we specify the location of the database file in which the results of the mining process will be stored and ensure the database does currently not exist. We then run the `mine_git_repo` function with the optional argument `max_modifications = 1`. With this only commits in which 1 or less files were modified are mined.

In [ ]:
import git2net

sqlite_db_file = 'git2net.db'

# Remove database if exists
if os.path.exists(sqlite_db_file):
    os.remove(sqlite_db_file)

max_modifications = 1
    
git2net.mine_git_repo(repo_dir, sqlite_db_file, max_modifications=max_modifications, timeout=1)

Found no database on provided path. Starting from scratch.


Parallel (8 processes):   1%|          | 1/128 [00:00<00:21,  5.93it/s]

Commit exceeding max_modifications:  bd0ad7b12500239321a8b7c6ba547f6111c781bb


Parallel (8 processes):   3%|▎         | 4/128 [00:00<00:31,  3.97it/s]

Timeout processing commit:  db0e20b413363cb93447ed4567bb8e959fc7f306
Timeout processing commit:  6a844a712042c3ce688a2060d8ae691b9ab86a32
Timeout processing commit:  ace737585423d53a9641069c486272ff0a16cd3d
Timeout processing commit:  c71a73528640d81b325e19aa38a3f68e4fe34366


...

  File "/usr/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
Parallel (8 processes):  94%|█████████▍| 120/128 [00:04<00:00, 36.24it/s]  File "/usr/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
KeyboardInterrupt


Commit exceeding max_modifications:  3f6435c4acbf8d1b6b02e9be02b514c2c2d53446
Commit exceeding max_modifications:  4de4c314a1261e784f1a54969a32ab4dfa439d76
Timeout processing commit:  56d841a65a39508b3325ed6b770ed20b405a61ec
Timeout processing commit:  51c8b566692857ed39e899316da79e6ce83b0c4d
Timeout processing commit:  f650aa008e91423a455c9303cc11dc5680944ce8


Parallel (8 processes):  98%|█████████▊| 125/128 [00:04<00:00, 36.72it/s]

Timeout processing commit:  3ca9e249a39dab08750eed286b2f16b1fb533fa1


Parallel (8 processes):  98%|█████████▊| 126/128 [00:19<00:00, 36.72it/s]

While mining, `git2net` provides information about the current progress. The first line shows that no database was found at the current path and mining will be started from scratch. This is totally expected, as we deliberately deleted any existing database before the run.

Subsequently, progress updates on the mining process are printed. The first information denotes the number of processes `git2net` spawns and runs on. `git2net` is highly parallelised and will automatically detect the number of threads of your CPU, fully utilising all of them during operation. In case you want to reduce this load, this can be done by specifically setting the number of processes with the `no_of_processes` option of the `mine_git_repo` function.

The other output shows the number of commits and total number of commits mined in this run, as well as the elapsed time and an estimate of the remaining time to finish.

If a commit is skipped, the reason and the commit hash are printed. Currently, there are three cases in which a commit can be skipped. Firstly, as seen above a commit can exceed the maximum number of modifications set by `max_modifications`. Secondly, processing the commit can take longer as a maximum time defined by the `timeout` option. Thirdly, a commit can be skipped due to an error occuring within the commit. In these cases, please report the repository and commit hash in a new issue on github.com/gotec/git2net.

Let's resume the mining process while increasing the maximum number of modifications to 5!

In [ ]:
max_modifications = 5

git2net.mine_git_repo(repo_dir, sqlite_db_file, max_modifications=max_modifications)

As you can see from the output above, the process was resumed from the old database, skipping the already processed commits in the repository.

Great, we made some progress and a large amount of the commits in the repository are already mined and in the database! But what about the other ones? We get some more information on the commits missing from the database from the `mining_state_summary` function. Similar to `mine_git_repo`, it also requires the paths to the repository as well as the database.

In [ ]:
git2net.mining_state_summary(repo_dir, sqlite_db_file)

The function again provides a summary of the mining state, as well as details on all missing commits. Let's assume, we are very interest in commit *090c00c342283134a23900f85c1d232499617365* but want to avoid crawling the other missing commits. While this is uneccessary for small repositories such as `git2net` this might become higly relevant for larger projects such as `linux`, where individual commits can make changes to thousands of files which in turn require significant computational resources to analyse. This is particularly important for merge commits, as all files included in the diffs to both parent commits need to be considered. Therefore, for larger projects I generally recommend to run `git2net` with `max_modifications = 1000`, subsequently increasing this number if required.

But now back to mining specifically commit *090c00c342283134a23900f85c1d232499617365*, which can be done with the `commits` option in `mine_git_repo`. We also set the number of processes to 1, enabling serial mode, which can be very helpful for debugging as significantly more information is printed.

In [ ]:
# mine_git_repo takes list of commits
commits = ['090c00c342283134a23900f85c1d232499617365']

git2net.mine_git_repo(repo_dir, sqlite_db_file, commits=commits, no_of_processes=1)

Congratulations, you have now mined your first git repository using `git2net`! Note, though that not all commits have been mined at this point. This will be done at a later stage of this tutorial.

## Visualisation and Analysis

You can now use the database to query various information on different commits or edits. In addition, `git2net` also provides the functionality to generate various network projections of the data.

To start, lets try to obtain a co-editing network for our project. This is as simple as calling the `get_coediting_network` function and providing the database we just mined.

In [ ]:
t, node_info, edge_info = git2net.get_coediting_network(sqlite_db_file)
t

The function returns a `pathpy` temporal network object as well as two dictionaries which can be used to return properties of nodes and edges. As of writing this tutorial not all of them are used but they are set as placeholders for future versions of `git2net`.

A `pathpy` temporal network object can be visualised by itself as shown above. In addition, we can also aggregate the network, by dropping the order of events, yielding a standard network object. Let's do this next.

In [ ]:
import pathpy as pp
pp.Network.from_temporal_network(t)

In both the temporal and aggregated network, a node represents an author, whereas edges point from the person changing a line of code to the person who was the original author.

Next, we could ask the question which those files were that authors collaborated on. Therefore, we can plot a bipartite network containing both files and authors as nodes.

In [ ]:
t, node_info, edge_info = git2net.get_bipartite_network(sqlite_db_file)
n = pp.Network.from_temporal_network(t)
n

For this network, `node_info` contains the classes of authors in the network. These can e.g. be used to color nodes as shown below.

In [ ]:
colour_map = {'author': '#73D2DE', 'file': '#2E5EAA'}
node_color = {node: colour_map[node_info['class'][node]] for node in n.nodes}
pp.visualisation.plot(n, node_color=node_color)

The projection of this network that links authors editing the same file is the co-authorship network.

In [ ]:
n, node_info, edge_info = git2net.get_coauthorship_network(sqlite_db_file)
n

Note that it looks similar, however, all information on the direction of interactions is lost.

If we are interested in e.g more recently edited files, we can filter the database by providing the `time_from` and `time_to` options. Let's check the files edited since May 2019.

In [ ]:
from datetime import datetime
time_from = datetime(2019, 5, 1)
t, node_info, edge_info = git2net.get_bipartite_network(sqlite_db_file, time_from=time_from)
n = pp.Network.from_temporal_network(t)
colour_map = {'author': '#73D2DE', 'file': '#2E5EAA'}
node_color = {node: colour_map[node_info['class'][node]] for node in n.nodes}
pp.visualisation.plot(n, node_color=node_color)

`git2net` allows the extraction of editing paths on the level of individual lines. I.e. we are able to track consecutive changes made to a single line over time&mdash;even if these lines move up or down in a file, or even across files. This is very powerful, as it allows us to determine editing sequences as well as find lines that require more editing than others. These could either be very difficult lines to implement or contain very important information, such as the version number in an `__init__.py` file.

To extract these paths, we can use the `get_line_editing_paths` function. As these networks tend be very large we limit the analysis to a very small file for this tutorial. To only look at a specific set of file paths we can use the `file_paths` option.

In [ ]:
paths, dag, node_info, edge_info = git2net.get_line_editing_paths(sqlite_db_file,
                                                                  file_paths=['git2net/__init__.py'])
pp.visualisation.plot(dag, node_color=node_info['colors'])

As you can see in the output above, the function first looks for aliases. These are other names of the files in the repository that can occur through renaming or moving the file. To follow the edits made to specific lines, we need to be aware of these renamings to track lines across these files.

Further notice, that despite only looking at a single file the network shown above is not connected. This is due to our database not being complete. Let's fix this now and try again.

In [ ]:
git2net.mine_git_repo(repo_dir, sqlite_db_file)

In [ ]:
paths, dag, node_info, edge_info = git2net.get_line_editing_paths(sqlite_db_file,
                                                                  file_paths=['git2net/__init__.py'])
pp.visualisation.plot(dag, node_color=node_info['colors'])

As mentioned before, these networks get very large very quickly. Therefore, it is often more useful to work with the `pathpy` path object that is also returned by the function. It cointains all paths and subpaths contained in the network shown above. More information regarding this object can be found in the documentation on [pathpy.net](http://www.pathpy.net/).

This concludes this tutorial, which I hope you found useful. Enjoy using `git2net` and best of luck for your research! If you find any bugs with the code please let me know on [github.com](https://github.com/gotec/git2net).

`git2net` has been developed as open source project. This means your ideas and inputs are highly welcome. Feel free to share the project and contribute yourself. You can imediately get started on the repository you just downloaded!